In [24]:
import pandas as pd
import requests
from urllib import parse

with open('APIkey.txt', 'r', encoding='utf-8') as file:
    apiKey=file.readline()
print(apiKey) # api키 : 매일 수정 필요
username = '장연석' # 유저아이디

id = parse.quote(username) # 아이디를 URL 인코딩

url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + id +'?api_key=' + apiKey #puuid값을 가져오기 위한 주소
r = requests.get(url)
r = r.json()


puuid = r['puuid'] # puuid값을 가져옴
r

RGAPI-c19c54d7-cd9a-4dda-a634-142b2f1e0b28


{'id': 'cvWysTa2FY5CUbTcIx9SHfWJIqwQzlhnODoFgl6mC8WxtQ',
 'accountId': 'B6TkJrk0cVf1GR0whPpPr2w_243nenT1rfQ5xYfPG02o',
 'puuid': 'oPNOg6lIz33RAo_pKPL_3kanTxNrqGYTqsCMuoLHajcmMOMqhWIT-K9sYB6jU4UOpgHya9afeXQayg',
 'name': '장연석',
 'profileIconId': 5936,
 'revisionDate': 1692162112000,
 'summonerLevel': 302}

In [25]:
n = str(10)
rankUrl = 'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/' + puuid + '/ids?queue=420&type=ranked&start=0&count='+n+'&api_key='+ apiKey
r = requests.get(rankUrl)
r = r.json()

rankId = r
print(rankId)

['KR_6654611881', 'KR_6654590955', 'KR_6654572270', 'KR_6654524849', 'KR_6654478830', 'KR_6654441170', 'KR_6654403331', 'KR_6654316576', 'KR_6644896404', 'KR_6644869633']


In [30]:
col_list=[]
def preprocessing(df,r_timeline,col_list=col_list):
    df1=df[df['summonerName']==username]
    df1['Player']=username
    df1['KDA 0.5']=(df1['kills']+df1['assists'])/(df1['deaths']+0.5)

    df1['CSM']=df1['totalMinionsKilled']/df1['timePlayed']

    df1['GPM']=df1['goldEarned']/df1['timePlayed']

    df1['VSPM']=df1['visionScore']/df1['timePlayed']
    df1['WPM']=df1['wardsPlaced']/df1['timePlayed']
    df1['VWPM']=df1['detectorWardsPlaced']/df1['timePlayed']
    df1['WCPM']=df1['wardsKilled']/df1['timePlayed']

    df1['DPM']=df1['totalDamageDealtToChampions']/df1['timePlayed']

    df1['K+A Per Minute']=(df1['kills']+df1['assists'])/df1['timePlayed']

    #이건 생략
    #  df1['Shutdown bounty collected']
    #  df1['Shutdown bounty lost']
    #  df1['Solo kills']=

    #timeline지표들
    usernum=str(df1['participantId'].iloc[0])
    oppnum=str((int(usernum)+5)%10)
    df1['GD@15']=r_timeline['info']['frames'][15]['participantFrames'][usernum]['totalGold']

    df1['CSD@15']=r_timeline['info']['frames'][15]['participantFrames'][usernum]['minionsKilled']

    df1['XPD@15']=r_timeline['info']['frames'][15]['participantFrames'][usernum]['xp']

    df1['LVLD@15']=r_timeline['info']['frames'][15]['participantFrames'][usernum]['level']-r_timeline['info']['frames'][15]['participantFrames'][oppnum]['level']#,'minionsKilled','xp']

    # %지표들

    df1['GOLD ratio']=df1['goldEarned']/df[df['teamId']==int(df1['teamId'])]['goldEarned'].sum(axis=0)*100
    df1['VS ratio']=df1['visionScore']/df[df['teamId']==int(df1['teamId'])]['visionScore'].sum(axis=0)*100
    df1['DMG ratio']=df1['totalDamageDealtToChampions']/df[df['teamId']==int(df1['teamId'])]['totalDamageDealtToChampions'].sum(axis=0)*100
    df1['KP ratio']=(df1['kills']+df1['assists'])/df[df['teamId']==int(df1['teamId'])]['kills'].sum(axis=0)*100

    return df1[col_list]

In [36]:
df_final

,Player,championName,teamPosition,champLevel,kills,deaths,assists,KDA 0.5,totalMinionsKilled,totalAllyJungleMinionsKilled,...,totalHeal,totalHealsOnTeammates,damageSelfMitigated,totalDamageShieldedOnTeammates,timeCCingOthers,totalTimeCCDealt,trueDamageTaken,totalTimeSpentDead,consumablesPurchased,itemsPurchased
0,장연석,Olaf,TOP,16,5,0,3,16.000000,229,4,...,9515,0,15146,0,12,347,2408,0,3,26
1,장연석,Sion,TOP,12,0,5,3,0.545455,124,0,...,2164,0,20136,0,28,380,351,157,2,13
2,장연석,XinZhao,JUNGLE,16,8,5,11,3.454545,45,83,...,18421,0,34006,0,26,568,1607,130,3,25
3,장연석,Khazix,JUNGLE,16,6,8,9,1.764706,40,98,...,13661,0,19454,0,14,367,817,157,5,26
4,장연석,Khazix,JUNGLE,16,14,7,6,2.666667,70,83,...,11969,0,21446,0,12,296,1735,221,3,24
5,장연석,Shaco,JUNGLE,13,5,7,1,0.800000,25,58,...,6093,0,11570,0,22,553,1158,146,3,23
6,장연석,Khazix,JUNGLE,10,3,2,4,2.800000,31,51,...,7011,0,5992,0,2,98,99,37,3,10
7,장연석,Orianna,MIDDLE,15,6,13,11,1.259259,174,0,...,221,0,22339,745,23,347,288,372,3,16
8,장연석,Khazix,JUNGLE,15,17,4,8,5.555556,13,86,...,12539,0,15546,0,8,171,410,78,3,23
9,장연석,Shaco,JUNGLE,18,21,6,8,4.461538,46,84,...,11756,0,24968,0,35,868,1035,229,1,20


In [35]:
df_final[['totalAllyJungleMinionsKilled','totalEnemyJungleMinionsKilled']]

,totalAllyJungleMinionsKilled,totalEnemyJungleMinionsKilled
0,4,17
1,0,0
2,83,9
3,98,0
4,83,16
5,58,4
6,51,2
7,0,2
8,86,3
9,84,33


In [31]:
df_final=pd.DataFrame()
with open('APIcols.txt', 'r', encoding='utf-8') as file:
    for line in file:
        col_list.append(line.strip()) 

for i in rankId:
    url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + i + '?api_key=' + apiKey
    r = requests.get(url)
    r = r.json()
    info = r['info']  # 전체 데이터에서 info를 추출
    part = info['participants'] # info 데이터에서 유저들의 정보 추출
    df= pd.DataFrame(part)
    url_timeline = f'https://asia.api.riotgames.com/lol/match/v5/matches/{i}/timeline?api_key={apiKey}'
    r_timeline=requests.get(url_timeline)
    r_timeline=r_timeline.json()
    df_final=pd.concat([df_final,preprocessing(df,r_timeline)])
    df_final = df_final.reset_index(drop=True)

C:\Users\066\AppData\Local\Temp\ipykernel_13496\2747204200.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Player']=username
C:\Users\066\AppData\Local\Temp\ipykernel_13496\2747204200.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['KDA 0.5']=(df1['kills']+df1['assists'])/(df1['deaths']+0.5)
C:\Users\066\AppData\Local\Temp\ipykernel_13496\2747204200.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [8]:
for i in rankId[:1]:
    url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + i + '?api_key=' + apiKey
    r = requests.get(url)
    r = r.json()
    info = r['info']  # 전체 데이터에서 info를 추출
    part = info['participants'] # info 데이터에서 유저들의 정보 추출
    df= pd.DataFrame(part)
    url_timeline = f'https://asia.api.riotgames.com/lol/match/v5/matches/{i}/timeline?api_key={apiKey}'
    r_timeline=requests.get(url_timeline)
    r_timeline=r_timeline.json()

In [9]:
df

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,challenges,champExperience,champLevel,...,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,0,0,3,0,0,0,5,"{'12AssistStreakCount': 0, 'abilityUses': 187,...",15903,16,...,3,5,1,0,0,18,1,3,7,True
1,0,0,6,0,1,0,2,"{'12AssistStreakCount': 0, 'abilityUses': 305,...",12664,14,...,3,3,1,0,0,27,4,3,5,True
2,0,0,5,0,0,0,0,"{'12AssistStreakCount': 0, 'abilityUses': 146,...",12365,14,...,0,1,1,0,0,13,1,1,7,True
3,0,2,10,0,0,0,0,"{'12AssistStreakCount': 0, 'abilityUses': 128,...",10705,13,...,2,4,1,0,0,25,3,5,10,True
4,0,7,17,0,0,0,0,"{'12AssistStreakCount': 1, 'abilityUses': 98, ...",8554,11,...,1,4,1,0,0,50,9,2,29,True
5,0,6,1,0,0,0,0,"{'12AssistStreakCount': 0, 'abilityUses': 252,...",8868,12,...,1,1,9,0,0,21,1,1,10,False
6,1,0,4,0,0,0,0,"{'12AssistStreakCount': 0, 'abilityUses': 348,...",8817,12,...,0,0,9,0,0,16,5,2,10,False
7,0,0,2,0,0,0,0,"{'12AssistStreakCount': 0, 'abilityUses': 107,...",10213,13,...,0,0,9,0,0,14,4,0,8,False
8,0,0,4,0,0,0,0,"{'12AssistStreakCount': 0, 'abilityUses': 135,...",9283,12,...,0,0,9,0,0,13,2,1,7,False
9,0,0,5,0,0,0,2,"{'12AssistStreakCount': 0, 'abilityUses': 98, ...",8512,11,...,0,0,9,0,0,18,0,3,8,False


In [15]:
r_timeline['info']['frames'][10]

{'events': [{'itemId': 3364,
   'participantId': 2,
   'timestamp': 540730,
   'type': 'ITEM_DESTROYED'},
  {'assistingParticipantIds': [7],
   'bounty': 0,
   'killerId': 10,
   'killerTeamId': 200,
   'monsterSubType': 'CHEMTECH_DRAGON',
   'monsterType': 'DRAGON',
   'position': {'x': 9866, 'y': 4414},
   'timestamp': 544961,
   'type': 'ELITE_MONSTER_KILL'},
  {'killerId': 0,
   'laneType': 'BOT_LANE',
   'position': {'x': 10504, 'y': 1029},
   'teamId': 100,
   'timestamp': 545729,
   'type': 'TURRET_PLATE_DESTROYED'},
  {'itemId': 3134,
   'participantId': 2,
   'timestamp': 549059,
   'type': 'ITEM_DESTROYED'},
  {'itemId': 1036,
   'participantId': 2,
   'timestamp': 549059,
   'type': 'ITEM_DESTROYED'},
  {'itemId': 6691,
   'participantId': 2,
   'timestamp': 549059,
   'type': 'ITEM_PURCHASED'},
  {'itemId': 2422,
   'participantId': 2,
   'timestamp': 550358,
   'type': 'ITEM_DESTROYED'},
  {'itemId': 3158,
   'participantId': 2,
   'timestamp': 550358,
   'type': 'ITEM_PUR

In [19]:
matching = [s for s in df.columns if "Ping" in s] 


In [20]:
matching

['allInPings',
 'assistMePings',
 'baitPings',
 'basicPings',
 'commandPings',
 'dangerPings',
 'enemyMissingPings',
 'enemyVisionPings',
 'getBackPings',
 'holdPings',
 'needVisionPings',
 'onMyWayPings',
 'pushPings',
 'visionClearedPings']

In [23]:
df[matching].sum(axis=1)

0     4
1    42
2     7
3    17
4    75
5    30
6    30
7    14
8    16
9    24
dtype: int64

In [41]:
df_final.to_csv('data.csv',index=False)

In [43]:
pd.read_csv("data.csv")

,Player,championName,teamPosition,champLevel,kills,deaths,assists,KDA 0.5,totalMinionsKilled,totalAllyJungleMinionsKilled,...,totalHeal.1,totalHealsOnTeammates.1,damageSelfMitigated.1,totalDamageShieldedOnTeammates.1,timeCCingOthers.1,totalTimeCCDealt.1,trueDamageTaken.1,totalTimeSpentDead.1,consumablesPurchased.1,itemsPurchased.1
0,장연석,Olaf,TOP,16,5,0,3,16.000000,229,4,...,9515,0,15146,0,12,347,2408,0,3,26
1,장연석,Sion,TOP,12,0,5,3,0.545455,124,0,...,2164,0,20136,0,28,380,351,157,2,13
2,장연석,XinZhao,JUNGLE,16,8,5,11,3.454545,45,83,...,18421,0,34006,0,26,568,1607,130,3,25
3,장연석,Khazix,JUNGLE,16,6,8,9,1.764706,40,98,...,13661,0,19454,0,14,367,817,157,5,26
4,장연석,Khazix,JUNGLE,16,14,7,6,2.666667,70,83,...,11969,0,21446,0,12,296,1735,221,3,24
5,장연석,Shaco,JUNGLE,13,5,7,1,0.800000,25,58,...,6093,0,11570,0,22,553,1158,146,3,23
6,장연석,Khazix,JUNGLE,10,3,2,4,2.800000,31,51,...,7011,0,5992,0,2,98,99,37,3,10
7,장연석,Orianna,MIDDLE,15,6,13,11,1.259259,174,0,...,221,0,22339,745,23,347,288,372,3,16
8,장연석,Khazix,JUNGLE,15,17,4,8,5.555556,13,86,...,12539,0,15546,0,8,171,410,78,3,23
9,장연석,Shaco,JUNGLE,18,21,6,8,4.461538,46,84,...,11756,0,24968,0,35,868,1035,229,1,20


In [44]:
import api
df= api.getAPI('장연석')

c:\Users\066\Desktop\FlyAI_Project\API\LeagueofLegends\api.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Player']=username
c:\Users\066\Desktop\FlyAI_Project\API\LeagueofLegends\api.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['KDA 0.5']=(df1['kills']+df1['assists'])/(df1['deaths']+0.5)
c:\Users\066\Desktop\FlyAI_Project\API\LeagueofLegends\api.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [45]:
df

,Player,championName,teamPosition,champLevel,kills,deaths,assists,KDA 0.5,totalMinionsKilled,totalAllyJungleMinionsKilled,...,totalHeal,totalHealsOnTeammates,damageSelfMitigated,totalDamageShieldedOnTeammates,timeCCingOthers,totalTimeCCDealt,trueDamageTaken,totalTimeSpentDead,consumablesPurchased,itemsPurchased
0,장연석,Olaf,TOP,16,5,0,3,16.000000,229,4,...,9515,0,15146,0,12,347,2408,0,3,26
1,장연석,Sion,TOP,12,0,5,3,0.545455,124,0,...,2164,0,20136,0,28,380,351,157,2,13
2,장연석,XinZhao,JUNGLE,16,8,5,11,3.454545,45,83,...,18421,0,34006,0,26,568,1607,130,3,25
3,장연석,Khazix,JUNGLE,16,6,8,9,1.764706,40,98,...,13661,0,19454,0,14,367,817,157,5,26
4,장연석,Khazix,JUNGLE,16,14,7,6,2.666667,70,83,...,11969,0,21446,0,12,296,1735,221,3,24
5,장연석,Shaco,JUNGLE,13,5,7,1,0.800000,25,58,...,6093,0,11570,0,22,553,1158,146,3,23
6,장연석,Khazix,JUNGLE,10,3,2,4,2.800000,31,51,...,7011,0,5992,0,2,98,99,37,3,10
7,장연석,Orianna,MIDDLE,15,6,13,11,1.259259,174,0,...,221,0,22339,745,23,347,288,372,3,16
8,장연석,Khazix,JUNGLE,15,17,4,8,5.555556,13,86,...,12539,0,15546,0,8,171,410,78,3,23
9,장연석,Shaco,JUNGLE,18,21,6,8,4.461538,46,84,...,11756,0,24968,0,35,868,1035,229,1,20


In [53]:
import pandas as pd
import requests
from urllib import parse

col_list=[]
username=''
with open('APIcols.txt', 'r', encoding='utf-8') as file:
    for line in file:
        col_list.append(line.strip()) 
print('XXXXXXX')
def preprocessing(df,r_timeline,col_list=col_list):
    df1=df[df['summonerName']==username]
    df1['Player']=username
    df1['KDA 0.5']=(df1['kills']+df1['assists'])/(df1['deaths']+0.5)

    df1['CSM']=df1['totalMinionsKilled']/df1['timePlayed']

    df1['GPM']=df1['goldEarned']/df1['timePlayed']

    df1['VSPM']=df1['visionScore']/df1['timePlayed']
    df1['WPM']=df1['wardsPlaced']/df1['timePlayed']
    df1['VWPM']=df1['detectorWardsPlaced']/df1['timePlayed']
    df1['WCPM']=df1['wardsKilled']/df1['timePlayed']

    df1['DPM']=df1['totalDamageDealtToChampions']/df1['timePlayed']

    df1['K+A Per Minute']=(df1['kills']+df1['assists'])/df1['timePlayed']

    #이건 생략
    #  df1['Shutdown bounty collected']
    #  df1['Shutdown bounty lost']
    #  df1['Solo kills']=

    #timeline지표들
    usernum=str(df1['participantId'].iloc[0])
    oppnum=str((int(usernum)+5)%10)
    df1['GD@15']=r_timeline['info']['frames'][15]['participantFrames'][usernum]['totalGold']

    df1['CSD@15']=r_timeline['info']['frames'][15]['participantFrames'][usernum]['minionsKilled']

    df1['XPD@15']=r_timeline['info']['frames'][15]['participantFrames'][usernum]['xp']

    df1['LVLD@15']=r_timeline['info']['frames'][15]['participantFrames'][usernum]['level']-r_timeline['info']['frames'][15]['participantFrames'][oppnum]['level']#,'minionsKilled','xp']

    # %지표들

    df1['GOLD ratio']=df1['goldEarned']/df[df['teamId']==int(df1['teamId'])]['goldEarned'].sum(axis=0)*100
    df1['VS ratio']=df1['visionScore']/df[df['teamId']==int(df1['teamId'])]['visionScore'].sum(axis=0)*100
    df1['DMG ratio']=df1['totalDamageDealtToChampions']/df[df['teamId']==int(df1['teamId'])]['totalDamageDealtToChampions'].sum(axis=0)*100
    df1['KP ratio']=(df1['kills']+df1['assists'])/df[df['teamId']==int(df1['teamId'])]['kills'].sum(axis=0)*100
    print(len(col_list))
    return df1[col_list]

def getAPI(name):
    N=10
    global username
    username=name
    with open('APIkey.txt', 'r', encoding='utf-8') as file:
        apiKey=file.readline()
    #username = '장연석'

    id = parse.quote(username) # 아이디를 URL 인코딩

    url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + id +'?api_key=' + apiKey #puuid값을 가져오기 위한 주소
    r = requests.get(url)
    r = r.json()

    puuid = r['puuid'] # puuid값을 가져옴

    n = str(N)
    rankUrl = 'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/' + puuid + '/ids?queue=420&type=ranked&start=0&count='+n+'&api_key='+ apiKey
    r = requests.get(rankUrl)
    r = r.json()

    rankId = r

    df_final=pd.DataFrame()
    for i in rankId:
        url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + i + '?api_key=' + apiKey
        r = requests.get(url)
        r = r.json()
        info = r['info']  # 전체 데이터에서 info를 추출
        part = info['participants'] # info 데이터에서 유저들의 정보 추출
        df= pd.DataFrame(part)
        url_timeline = f'https://asia.api.riotgames.com/lol/match/v5/matches/{i}/timeline?api_key={apiKey}'
        r_timeline=requests.get(url_timeline)
        r_timeline=r_timeline.json()
        df_final=pd.concat([df_final,preprocessing(df,r_timeline)])
        df_final = df_final.reset_index(drop=True)
    return df_final

XXXXXXX


In [54]:
df1=getAPI('장연석')

C:\Users\066\AppData\Local\Temp\ipykernel_13496\3476780416.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Player']=username
C:\Users\066\AppData\Local\Temp\ipykernel_13496\3476780416.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['KDA 0.5']=(df1['kills']+df1['assists'])/(df1['deaths']+0.5)
C:\Users\066\AppData\Local\Temp\ipykernel_13496\3476780416.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

108


C:\Users\066\AppData\Local\Temp\ipykernel_13496\3476780416.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Player']=username
C:\Users\066\AppData\Local\Temp\ipykernel_13496\3476780416.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['KDA 0.5']=(df1['kills']+df1['assists'])/(df1['deaths']+0.5)
C:\Users\066\AppData\Local\Temp\ipykernel_13496\3476780416.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

108


C:\Users\066\AppData\Local\Temp\ipykernel_13496\3476780416.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Player']=username
C:\Users\066\AppData\Local\Temp\ipykernel_13496\3476780416.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['KDA 0.5']=(df1['kills']+df1['assists'])/(df1['deaths']+0.5)
C:\Users\066\AppData\Local\Temp\ipykernel_13496\3476780416.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

...

C:\Users\066\AppData\Local\Temp\ipykernel_13496\3476780416.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Player']=username
C:\Users\066\AppData\Local\Temp\ipykernel_13496\3476780416.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['KDA 0.5']=(df1['kills']+df1['assists'])/(df1['deaths']+0.5)
C:\Users\066\AppData\Local\Temp\ipykernel_13496\3476780416.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

108


C:\Users\066\AppData\Local\Temp\ipykernel_13496\3476780416.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Player']=username
C:\Users\066\AppData\Local\Temp\ipykernel_13496\3476780416.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['KDA 0.5']=(df1['kills']+df1['assists'])/(df1['deaths']+0.5)
C:\Users\066\AppData\Local\Temp\ipykernel_13496\3476780416.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

108
108


C:\Users\066\AppData\Local\Temp\ipykernel_13496\3476780416.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Player']=username
C:\Users\066\AppData\Local\Temp\ipykernel_13496\3476780416.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['KDA 0.5']=(df1['kills']+df1['assists'])/(df1['deaths']+0.5)
C:\Users\066\AppData\Local\Temp\ipykernel_13496\3476780416.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [48]:
df1

,Player,championName,teamPosition,champLevel,kills,deaths,assists,KDA 0.5,totalMinionsKilled,totalAllyJungleMinionsKilled,...,totalHeal,totalHealsOnTeammates,damageSelfMitigated,totalDamageShieldedOnTeammates,timeCCingOthers,totalTimeCCDealt,trueDamageTaken,totalTimeSpentDead,consumablesPurchased,itemsPurchased
0,장연석,Olaf,TOP,16,5,0,3,16.000000,229,4,...,9515,0,15146,0,12,347,2408,0,3,26
1,장연석,Sion,TOP,12,0,5,3,0.545455,124,0,...,2164,0,20136,0,28,380,351,157,2,13
2,장연석,XinZhao,JUNGLE,16,8,5,11,3.454545,45,83,...,18421,0,34006,0,26,568,1607,130,3,25
3,장연석,Khazix,JUNGLE,16,6,8,9,1.764706,40,98,...,13661,0,19454,0,14,367,817,157,5,26
4,장연석,Khazix,JUNGLE,16,14,7,6,2.666667,70,83,...,11969,0,21446,0,12,296,1735,221,3,24
5,장연석,Shaco,JUNGLE,13,5,7,1,0.800000,25,58,...,6093,0,11570,0,22,553,1158,146,3,23
6,장연석,Khazix,JUNGLE,10,3,2,4,2.800000,31,51,...,7011,0,5992,0,2,98,99,37,3,10
7,장연석,Orianna,MIDDLE,15,6,13,11,1.259259,174,0,...,221,0,22339,745,23,347,288,372,3,16
8,장연석,Khazix,JUNGLE,15,17,4,8,5.555556,13,86,...,12539,0,15546,0,8,171,410,78,3,23
9,장연석,Shaco,JUNGLE,18,21,6,8,4.461538,46,84,...,11756,0,24968,0,35,868,1035,229,1,20
